# Monadic functions

## Outline

* What are they

* Examples of monadic functions

In this lesson, we will learn about monadic functions and how you can use them.

## What are they

For the start let's write out some short definitions that we already know:
- A **monad** is a typeclass that defines `>>=` and `return` (minimally).

- A **monadic value** is a value contextualized by a type constructor that has an instance of the monad type class.

- The **inner value** of a monadic value is the actual value that was contextualized by a type constructor.

That brings us to the definition of a **monadic function**:
- A **monadic function** is a function that produces a monadic value. 

- **Monadic code** is code defined inside of a monadic function.

This is a very broad definition. We can say all functions that we wrote so far which returned `IO a`, `Maybe a` and `[a]`, can be called monadic functions. 

Further we distinguish between two types of monadic functions:
- **Classic monadic functions** are in the form `f :: a -> m b`, where a is the type of the inner value of the monad.

- **Loose monadic functions** are in the form `f :: anything -> m b`, where the input of the function really doesn't matter.

What we will be focusing on in this chapter are some of the monadic functions which are available from Haskell modules and make writing monadic code easier.

## Examples of monadic functions

Here we cover some of the often used available monadic functions, explain what they do and when to use them.

All of the functions presented below are contained in the **Control.Monad** module.

### Lifting functions

The first of the lifting functions is the `liftM` function. Its definition is:
```haskell
liftM :: Monad m => (a1 -> r) -> m a1 -> m r 
liftM f m1 = do 
    x1 <- m1
    return (f x1) 
```

It promotes a function to a monad. Let's look at an example.

In [ ]:
import Control.Monad (liftM)

add1 :: Num a => a -> a
add1 x1 = x1 + 1

var1 :: Maybe Int
var1 = Just 2

result1 :: Maybe Int
result1 = liftM add1 var1

We notice that we could get the same result if we used the Functor operator `fmap` instead of `liftM`.

The reason why `liftM` is usefull is because it is expressed with the monad operators `>>=` and `return`. 

If we define an instance of monad for a type we created we also have to define an instance of Applicative and Functor.

And in the functor definition we can simply say that `fmap = liftM`. 

In case we have a function that takes in two arguments we can use the `liftM2` function.
```haskell
liftM2 :: Monad m => (a1 -> a2 -> r) -> m a1 -> m a2 -> m r 
liftM2 f m1 m2 = do 
    x1 <- m1 
    x2 <- m2 
    return (f x1 x2)
```

In [ ]:
liftM2 (+) [0,1] [0,2]
liftM2 (+) (Just 1) Nothing

Also the functions `liftM3`, `liftM4` and `liftM5` exist. The numbers represent the number of arguments that their input functions accept.

The last of the lifting functions is the `ap` function.
```haskell
ap :: Monad m => m (a -> b) -> m a -> m b 
ap m1 m2 = do 
    f1 <- m1 
    x1 <- m2 
    return (f1 x1)
```

It works the same as the `<*>` operator from Applicative type class. It is useful for the same reason as the `liftM` function.

If we define an instance of monad for our type, we also have to define an instance of applicative and can say `(<*>) = ap`. 

### The `sequence` and `sequence_` functions

The definition of the `sequence` function is:
```haskell
sequence :: (Traversable t, Monad m) => t (m a) -> m (t a)
```

It evaluates each monadic action in the traversable structure from left to right, and collects the results.

Here is an example of this function:

In [ ]:
import Control.Monad (sequence)

var1 :: Maybe [Int]
var1 = Just [1,2,3,4]

list1 :: [Maybe Int]
list1 = sequence var1

We could also define a function that works reversed as the `sequence` function for lists.

In [ ]:
import Control.Applicative (liftA2)

sequenceR :: (Applicative f) => [f a] -> f [a]
sequenceR = foldr (liftA2 (:)) (pure [])

list2Var :: Maybe [Int]
list2Var = sequenceR list1

The `liftA2` function from the **Control.Applicative** module basically works the same as the `liftM2` function.
```haskell
liftA2 :: (a -> b -> c) -> f a -> f b -> f c 
```
The difference is that it promotes a function to an applicative instead of a monad.

The function `sequence_` works similar as `sequence` just that it throws away the result.
```haskell
sequence_ :: (Foldable t, Monad m) => t (m a) -> m () 
```

This comes useful when you need to get the monad context out of the variable.

For instance in the code example below we use it to be able to print elements of a list one by one.

In [ ]:
import Control.Monad (sequence_)

mapPrint :: Show a => [a] -> IO ()
mapPrint xs = sequence_ . map print $ xs

mapPrint [1..3]

If we would not use the `sequence_` function we would get a compile error.

In Haskell it is a general convention if a function has an underscore at the end it means that it throws away the result. 

### The `mapM` and `mapM_` functions

The type signature of these functions are:
```haskell
mapM :: (Traversable t, Monad m) => (a -> m b) -> t a -> m (t b) 

mapM_ :: (Foldable t, Monad m) => (a -> m b) -> t a -> m () 
```

## Recap

In this lesson we've discussed:

- the definition of monadic functions

- examples of monadic functions